In [1]:
#importamos las Librerias
import gzip
import pandas as pd
import json
import ast
import pyarrow as pa
import pyarrow.parquet as pq

Leemos los Archivos

In [2]:
# Usamos with para abrir el gzip y asegurar que se cierre correctamente leemos las lineas y eliminamos los espacios, 
#para luego cargarlo en un dataframe
with gzip.open("steam_games.json.gz", "rt", encoding="utf-8") as g:
    data_s_games = [line.strip() for line in g]
df_s_games = pd.read_json("[" + ",".join(data_s_games) + "]")

In [80]:
#Hacemos head para tener una vista rapida del df
df_s_games.head()

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
0,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
1,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
2,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
3,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None
4,None,None,None,None,None,None,None,None,None,None,NaN,NaN,None


In [81]:
df_s_games.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120445 entries, 0 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 11.9+ MB


Podemos notar que existe gran cantidad de datos nulos por lo que procedemos a eliminar las filas que tienen todas las entradas con nulos
y actualizamos el dataframe

In [3]:
# usamos dropna para eliminar los nulos con la condiciom 'how' garantizamos que se elimine los que tienen todas entradas nulas 
df_s_games.dropna(inplace=True, how='all')

In [83]:
df_s_games.info()

<class 'pandas.core.frame.DataFrame'>
Index: 32135 entries, 88310 to 120444
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   publisher     24083 non-null  object 
 1   genres        28852 non-null  object 
 2   app_name      32133 non-null  object 
 3   title         30085 non-null  object 
 4   url           32135 non-null  object 
 5   release_date  30068 non-null  object 
 6   tags          31972 non-null  object 
 7   reviews_url   32133 non-null  object 
 8   specs         31465 non-null  object 
 9   price         30758 non-null  object 
 10  early_access  32135 non-null  float64
 11  id            32133 non-null  float64
 12  developer     28836 non-null  object 
dtypes: float64(2), object(11)
memory usage: 3.4+ MB


In [84]:
df_s_games.head(2)


,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer
88310,Kotoshiro,"[Action, Casual, Indie, Simulation, Strategy]",Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro
88311,"Making Fun, Inc.","[Free to Play, Indie, RPG, Strategy]",Ironbound,Ironbound,http://store.steampowered.com/app/643980/Ironb...,2018-01-04,"[Free to Play, Strategy, Indie, RPG, Card Game...",http://steamcommunity.com/app/643980/reviews/?...,"[Single-player, Multi-player, Online Multi-Pla...",Free To Play,0.0,643980.0,Secret Level SRL


In [4]:
df_s_games['release_date']

88310     2018-01-04
88311     2018-01-04
88312     2017-07-24
88313     2017-12-07
88314           None
             ...    
120440    2018-01-04
120441    2018-01-04
120442    2018-01-04
120443    2017-09-02
120444          None
Name: release_date, Length: 32135, dtype: object

In [5]:
# Extraer el año de la columna "release_date" y reemplazar los valores nulos con "sin dato
# "
df_s_games['Año'] = df_s_games['release_date'].str.extract(r'(\d{4})').fillna('sin dato')



In [6]:
df_s_games["genres"][88310]

['Action', 'Casual', 'Indie', 'Simulation', 'Strategy']

In [10]:
# desanidamos los valores de tipo lista que se encuentran en la columna 'genres'. Para eso utilizamos la función .explode
df_s_games = df_s_games.explode('genres')
df_s_games.head(5)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,Año
88310,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,2018
88310,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,2018
88310,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,2018
88310,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,2018
88310,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,2018


In [12]:
df_s_games ["id"]= df_s_games["id"].astype (str, errors="ignore")
df_s_games.head(1)

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,specs,price,early_access,id,developer,Año
88310,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,[Single-player],4.99,0.0,761140.0,Kotoshiro,2018


In [13]:
guardar = "steam_games.csv"

df_s_games.to_csv(guardar, index=False, encoding="utf-8")

In [15]:
#Transformo el archivo csv a parquet
#Leo el archivo csv
steam_games= pd.read_csv("steam_games.csv") 

#Indico donde quiero guardar el parquet y con que nombre
output_file= "steam_games.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(steam_games)
pq.write_table(table,output_file)